# 🎯 Week 5-6, Notebook 6: Few-Shot Learning for Actionable Insights

**Module:** LLMs, Prompt Engineering & RAG  
**Project:** Build the Knowledge Core for the Manufacturing Copilot

---

Fine-tuning an entire Large Language Model on your custom data is a powerful but often expensive and time-consuming process. **Few-Shot Learning** offers a compelling and highly effective alternative. The core idea is to guide a pre-trained LLM by providing a handful of high-quality examples (`shots`) directly within the prompt itself. This technique, also known as **in-context learning**, allows the model to learn a new task or adapt to a new domain on the fly.

In this notebook, we will use few-shot learning to build a system that suggests corrective actions for machine operators based on incident reports. This is a core feature of our Manufacturing Copilot, demonstrating how we can elicit expert-level behavior from a general-purpose model without any changes to its weights.

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:

1.  **Distinguish Few-Shot from Zero-Shot:** Articulate when and why providing in-context examples is necessary to improve model performance.
2.  **Curate High-Quality Exemplars:** Understand the importance of creating a small, clean, and representative dataset of examples (known as exemplars) that reflect real-world problems and solutions.
3.  **Automate Example Selection with Semantic Search:** Implement a system that uses semantic similarity to dynamically find the most relevant examples from your dataset for any new, incoming query.
4.  **Build and Evaluate a Few-Shot System:** Combine prompt templates, a curated exemplar database, and a semantic retrieval mechanism to generate useful, actionable, and contextually relevant recommendations.

## ⚙️ Setup: The Model and the Exemplar Dataset

We will continue to use the `google/flan-t5-base` model for its excellent balance of performance and resource efficiency.

The most critical component of a few-shot learning system is the **exemplar dataset**. This is a small, carefully curated collection of high-quality input-output pairs that represent the task we want the model to perform. For our use case, this will be a set of incident tickets and the corresponding corrective actions taken by experienced engineers. The quality of these exemplars is paramount—garbage in, garbage out.

# Hands-On: Initializing the Model and Creating the Exemplar Dataset
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
import pandas as pd
import torch

# --- Model Initialization ---
device = 0 if torch.cuda.is_available() else -1
model_name = "google/flan-t5-base"

try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    generator = pipeline(
        'text2text-generation',
        model=model,
        tokenizer=tokenizer,
        device=device,
        max_length=128,  # Corrective actions should be concise
        temperature=0.1,
    )
    print(f"Pipeline created successfully for model '{model_name}'.")
except Exception as e:
    print(f"Failed to create pipeline: {e}")
    generator = lambda x: [{"generated_text": "Error: Pipeline not initialized."}]


# --- Exemplar Dataset Creation ---
# This is our curated dataset of high-quality examples. In a real system,
# this would be stored in a database or a version-controlled file.
exemplars_df = pd.DataFrame([
    {"ticket": "Vibration spike on compressor #7 after bearing replacement.", "action": "Inspect bearing seating and alignment. Re-torque all mounting fasteners to spec."},
    {"ticket": "Hydraulic leak detected on the main clamp cylinder of Press-3.", "action": "Immediately isolate the machine using LOTO protocol. Replace the primary rod and piston seals."},
    {"ticket": "Camera on the main SMT line is misreading 2D barcodes due to glare from overhead lights.", "action": "Adjust the angle and polarization of the overhead lighting. Recalibrate the vision system's exposure and contrast thresholds."},
    {"ticket": "The packing line robot #2 is flagging repeated overcurrent alarms on its wrist axis (J5).", "action": "Verify the robot's payload is within specification. Check for mechanical binding and recalibrate motor torque limits."},
    {"ticket": "AGV-5 is slowing down and reporting intermittent lidar faults when approaching charging station B.", "action": "Clean the lidar sensor lens with an approved solution. Check the station's firmware for pending updates."}
])

print("\n--- Curated Exemplar Dataset ---")
exemplars_df

## 🛠️ Part 1: Building a Static Few-Shot Prompt

The core idea of few-shot learning is to construct a prompt that includes both the instructions for the task and the high-quality examples. The LLM then uses these examples to understand the desired input-output pattern and applies that pattern to the new query.

A well-structured few-shot prompt has three key parts:
1.  **Instruction:** A clear, high-level command telling the model what to do.
2.  **Exemplars:** A few `input -> output` pairs, formatted consistently.
3.  **Query:** The new input for which we want a response, followed by a trigger for the model to start generating.

In [ ]:
# Hands-On: Constructing a Few-Shot Prompt
def build_few_shot_prompt(examples_df, query):
    """Constructs a complete few-shot prompt with instructions, examples, and a query."""

    # 1. The high-level instruction
    instruction = "You are a senior reliability engineer. Based on the incident ticket, recommend a single, concise corrective action for the maintenance team."

    # 2. The formatted exemplars
    exemplar_texts = []
    for _, row in examples_df.iterrows():
        exemplar_texts.append(f"Ticket: {row['ticket']}\nAction: {row['action']}")

    # 3. The new query, formatted to match the exemplars
    query_text = f"Ticket: {query}\nAction:"

    # Combine all parts with a clear separator
    return "\n===\n".join([instruction] + exemplar_texts + [query_text])

# --- Test the prompt construction ---
# Let's create a prompt using the first 3 exemplars from our dataset.
new_incident_ticket = "The coolant pump for CNC-12 is showing a gradual pressure drop over the last 3 hours, but there are no visible leaks."
static_few_shot_prompt = build_few_shot_prompt(exemplars_df.head(3), new_incident_ticket)

print("--- Generated Static Few-Shot Prompt ---")
print(static_few_shot_prompt)

# --- Generate a response using the static prompt ---
print("--- LLM Response (Static Few-Shot) ---")
response = generator(static_few_shot_prompt)
print(response[0]['generated_text'])

## 🤖 Part 2: Dynamic Exemplar Selection with Semantic Search

Manually picking which examples to include in the prompt is not a scalable or effective solution. For a real-world system, we need a way to **automatically find the most relevant examples** for any given ticket. The best examples are those that are semantically similar to the new problem. We can achieve this using **semantic search**.

The process is straightforward:
1.  **Embed the Knowledge:** Convert all of our high-quality exemplar tickets into numerical vectors (embeddings) using a sentence transformer model. This is a one-time, offline process.
2.  **Embed the Query:** When a new ticket arrives, convert its text into an embedding using the same model.
3.  **Compare and Select:** Calculate the similarity (typically using cosine similarity) between the new ticket's embedding and all the exemplar embeddings. Select the top-k most similar exemplars.

This ensures that the examples provided to the LLM are highly relevant to the problem at hand, dramatically improving the quality of the generated response.

In [ ]:
# Hands-On: Implementing Dynamic Exemplar Selection
# We need to install the sentence-transformers library for this step.
# In a real project, you would add this to your requirements.txt file.
try:
    from sentence_transformers import SentenceTransformer, util
except ImportError:
    print("SentenceTransformers library not found. Installing...")
    import sys
    !{sys.executable} -m pip install -q sentence-transformers
    from sentence_transformers import SentenceTransformer, util

# --- Step 1: Embed the Exemplar Corpus ---
# Use a lightweight but effective model for creating the embeddings.
# "all-MiniLM-L6-v2" is a great choice for general-purpose semantic search.
embedding_model = SentenceTransformer("all-MiniLM-L6-v2", device=device)

# This is a one-time process. In a real system, you'd store these embeddings.
corpus_embeddings = embedding_model.encode(exemplars_df['ticket'].tolist(), convert_to_tensor=True)
print(f"Successfully created {corpus_embeddings.shape[0]} embeddings of dimension {corpus_embeddings.shape[1]}.")

# --- Step 2: Embed the New Query ---
query_embedding = embedding_model.encode(new_incident_ticket, convert_to_tensor=True)

# --- Step 3: Find the Most Similar Exemplars ---
# Calculate cosine similarity between the query and all corpus embeddings.
similarities = util.cos_sim(query_embedding, corpus_embeddings).squeeze()

# Add the similarity scores to our dataframe and sort to find the best matches.
exemplars_df['similarity'] = similarities.cpu().numpy()
retrieved_exemplars_df = exemplars_df.sort_values("similarity", ascending=False)

print(f"\n--- Most Relevant Exemplars for Query: '{new_incident_ticket}' ---")
retrieved_exemplars_df

Now, instead of using static, hardcoded examples, we can build our prompt using the **best examples** found by our semantic search retriever. This makes the prompt highly adaptive and context-aware.

In [ ]:
# Hands-On: Generating a Response with a Dynamic Prompt
# Select the top k most similar exemplars.
top_k = 3
best_exemplars_df = retrieved_exemplars_df.head(top_k)

# Build a new prompt with these dynamically selected, highly relevant examples.
dynamic_few_shot_prompt = build_few_shot_prompt(best_exemplars_df, new_incident_ticket)

print("--- Dynamically Generated Few-Shot Prompt (Top 3 Exemplars) ---")
print(dynamic_few_shot_prompt)

# --- Generate the final response ---
print("\n--- LLM Response (Dynamic Few-Shot) ---")
dynamic_response = generator(dynamic_few_shot_prompt)
print(dynamic_response[0]['generated_text'])

## ✅ Summary and Next Steps

This notebook demonstrated the power and elegance of few-shot learning with dynamic exemplar selection. By providing a small number of relevant, high-quality examples directly in the prompt, we can guide a general-purpose LLM to perform a specific, domain-intensive task with impressive accuracy—all without the need for costly fine-tuning.

**Key Takeaways:**

-   **Quality Over Quantity:** A few well-chosen, high-quality examples are far more effective than many noisy or irrelevant ones. Your exemplar dataset is a critical asset.
-   **Relevance is Key:** Static, hardcoded examples are brittle. A dynamic approach using semantic search to find the most relevant exemplars for any given query is crucial for building a robust and scalable system.
-   **A Systemic Approach:** A powerful few-shot learning system is more than just a prompt. It's a combination of a clear prompt template, a curated exemplar database, and an efficient retrieval mechanism working together.

In the next notebook, we will explore the fundamentals of **Retrieval-Augmented Generation (RAG)**. RAG takes the retrieval idea a step further, allowing us to fetch information not just from a small set of exemplars, but from a vast knowledge base of documents, such as technical manuals, past incident reports, or safety procedures. This is the final step in building a truly knowledgeable and trustworthy Manufacturing Copilot.